In [ ]:
import numpy as np
import pandas as pd
import re, html, unicodedata, warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LeakyReLU, Dropout, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow
tensorflow.keras.backend.set_epsilon(1)
np.random.seed(0)
from lightgbm import LGBMRegressor

In [ ]:
main_train = train_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
main_test = train_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
#main_train = train_df = pd.read_csv('train.csv')
#main_test = train_df = pd.read_csv('test.csv')

In [ ]:
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}
import re
from string import digits

totalMaxWordsCount = 100

def clean(text):
    text = re.sub(r'[^\w\s]','', text)
    text = re.sub("\n","", text)
    text = text.replace("'", "")
    text = text.replace("", "")
    remove_digits = text.maketrans('', '', digits)
    text = text.translate(remove_digits)
    
    text = text.lower()
    return text

def getMinMaxAvgWordsCountFromDf(df):
    maxWordsCount = 0
    minWordsCount = 10000000
    totalWordsCount = 0
    for i in range(0, len(df)):
        words = train_df.iloc[i,1].split()
        totalWordsCount = len(words) + totalWordsCount
        if len(words) > maxWordsCount:
            maxWordsCount = len(words)
        if len(words) < minWordsCount:
            minWordsCount = len(words)
    return minWordsCount, maxWordsCount, totalWordsCount/len(df)

global ENCODED_WORDS
global ENCODED_WORDS_COUNT
ENCODED_WORDS = {}
ENCODED_WORDS_COUNT = 0

def getEncodedWordsCount():
    return ENCODED_WORDS_COUNT

def setEncodedWordsCount():
    global ENCODED_WORDS_COUNT
    ENCODED_WORDS_COUNT = ENCODED_WORDS_COUNT + 1
    return ENCODED_WORDS_COUNT

def cleanWords(words, is_train):
    newWords = []
    global encodedWordsCount
    global ENCODED_WORDS
    global ENCODED_WORDS_COUNT
    encodedWordsCount = getEncodedWordsCount()
    for i in range(0, 150):
        try:
            
            if words[i] not in stop_words:
                needWord = words[i]
                try:
                    encodedWord = ENCODED_WORDS[needWord]
                    newWords.append(encodedWord)
                except KeyError:
                    if is_train == True:
                        ENCODED_WORDS[needWord] = encodedWordsCount
                        encodedWordsCount = setEncodedWordsCount()
                        newWords.append(ENCODED_WORDS[needWord])
                    else:
                        newWords.append(0)
            else:
                newWords.append(0)
                                    
        except IndexError:
            newWords.append(0)
    return newWords

In [ ]:
train_df = main_train
train_df = train_df.drop_duplicates(subset=['id', 'target', 'excerpt'], keep='last')
train_df['clean_text'] = train_df['excerpt'].apply(lambda x: clean(x))
train_df = train_df.drop(['url_legal', 'license', 'excerpt', 'standard_error'], axis = True)
train_df = train_df.drop(['id'], axis = True)

In [ ]:
X_train = []
for i in range(0, len(train_df)):
    words = train_df.iloc[i,1].split()
    words = cleanWords(words, True)
    X_train.append(words)

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
df = train_df
df = df.fillna(0)
df = df.drop(['clean_text'], axis = True)

In [ ]:
Y = df[['target']]
df = df.drop(['target'], axis=True)
X = X_train
Y = Y['target'].tolist()
#Y = np.asarray(Y)
X = np.asarray(X)

In [ ]:
X = X.astype(float)
for i in range(0, len(Y)):
    Y[i] = [Y[i]]

In [ ]:
len(X), len(Y)

In [ ]:
from tensorflow.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

#Y_scaler = min_max_scaler.fit_transform(Y.values)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)


min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(X_train)
X_train = min_max_scaler.transform(X_train)
X_val = min_max_scaler.transform(X_val)

#Y_train = Y_train.reshape(-1,1)
#Y_val = Y_val.reshape(-1,1)

min_max_scaler_Y = preprocessing.MinMaxScaler()
min_max_scaler_Y = min_max_scaler_Y.fit(Y_train)
Y_train = min_max_scaler_Y.transform(Y_train)
Y_val = min_max_scaler_Y.transform(Y_val)

# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
# X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [ ]:
num_units = 100
activation_function = 'sigmoid'
batch_size = 8
num_epochs = 200

# Initialize the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(Dense(1000, activation='tanh'))



regressor.add(Dense(1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mse', metrics = [root_mean_squared_error])

# Using the training set to train the model
history = regressor.fit(X_train, Y_train, epochs=150, batch_size= 50, verbose=2, validation_data=(X_val, Y_val))

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['root_mean_squared_error'])
pyplot.show()

In [ ]:
test_acc = regressor.evaluate(X_val, Y_val, verbose=2)
test_acc

In [ ]:
import math
def rmse_metric(predicted, actual):
    error = sum((b - a) ** 2 for a,b in zip(actual, predicted))
    n = len(actual)
    rmse = math.sqrt(error * 1.0 / n)
    return rmse
r2_regular = rmse_metric(Y_val, regressor.predict(X_val))

r2_regular

In [ ]:
regressor.evaluate(X_train, Y_train)

In [ ]:
ddd = main_test
ddd = ddd.drop_duplicates(subset=['id', 'excerpt'], keep='last')
ddd['clean_text'] = ddd['excerpt'].apply(lambda x: clean(x))
ddd = ddd.drop(['url_legal', 'license', 'excerpt', 'id'], axis = True)

In [ ]:
ddd_X_train = []
for i in range(0, len(ddd)):
    words = ddd.iloc[i,0].split()
    words = cleanWords(words, False)
    ddd_X_train.append(words)

In [ ]:
ddd_X_train = np.asarray(ddd_X_train)
ddd_X_train = ddd_X_train.astype(float)
ddd_X_train = min_max_scaler.transform(ddd_X_train)

In [ ]:
ddd_X_train

In [ ]:
predicted_targets = regressor.predict(ddd_X_train)

In [ ]:
predicted_targets

In [ ]:

predicted_targets_inverse = min_max_scaler_Y.inverse_transform(predicted_targets)
predicted_targets = predicted_targets_inverse

In [ ]:
predicted_targets

In [ ]:
finish = main_test
finish = finish.drop(['url_legal', 'license', 'excerpt'], axis = True)

In [ ]:
for i in range(0, len(finish)):
    finish.loc[i, 'target'] = predicted_targets[i][0]

In [ ]:
finish

In [ ]:
finish.to_csv('submission.csv', index = False)